In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1" #model will be trained on GPU 1

In [ ]:
#!pip install seaborn
import seaborn as sns

In [ ]:
import os
import cv2
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K

Using TensorFlow backend.


In [ ]:
import numpy as np
import scipy.misc
import numpy.random as rng
from PIL import Image, ImageDraw, ImageFont
from sklearn.utils import shuffle
#!pip install nibabel
import nibabel as nib #reading MR images
from sklearn.model_selection import train_test_split
import math
import glob
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
ff = glob.glob('E:/Gm_yes_new/*')

In [ ]:
len(ff)

330

In [ ]:
ff[0]

'E:/Gm_yes_new\\S1-1-0001.nii'

In [ ]:
gg = glob.glob('E:/Gm_no_new/*')

In [ ]:
len(gg)

299

In [ ]:
list=ff+gg

In [ ]:
len(list)

629

In [ ]:
new_list = np.asarray(list)

In [ ]:
images = []
Y=[]
for f in range(len(ff)):
    a = nib.load(ff[f])
    a = a.get_data()
    a = a[:,78:129,:]
    for i in range(a.shape[1]):
        Y.append("Yes")
        images.append((a[:,i,:]))
print (a.shape)

C:\Users\Dell\anaconda3\envs\labelme\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  """
C:\Users\Dell\anaconda3\envs\labelme\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  """
C:\Users\Dell\anaconda3\envs\labelme\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data

(121, 51, 121)


C:\Users\Dell\anaconda3\envs\labelme\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  """


In [ ]:
len(Y)

16830

In [ ]:
#images = np.asarray(images)

In [ ]:
len(images)

16830

In [ ]:
print(images[0].shape)

(121, 121)


In [ ]:
images1 = []
Y1=[]
for f in range(len(gg)):
    a = nib.load(gg[f])
    a = a.get_data()
    a = a[:,78:129,:]
    for i in range(a.shape[1]):
        Y1.append("No")
        images1.append((a[:,i,:]))
print (a.shape)

C:\Users\Dell\anaconda3\envs\labelme\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  """
C:\Users\Dell\anaconda3\envs\labelme\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  """
C:\Users\Dell\anaconda3\envs\labelme\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data

(121, 51, 121)


C:\Users\Dell\anaconda3\envs\labelme\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  """


In [ ]:
print(a.shape)

(121, 51, 121)


In [ ]:
images1[2].shape #2d image

(121, 121)

In [ ]:
#we need to have a 2D input for the SVC model. So you need to flatten your 3D images and create a feature matrix with dimension [# images, # pixels in image].

In [ ]:
#Feature extraction in the case of SVMs is really important. Thus, we start off initially with feature extraction.

In [ ]:
images_new=images+images1

In [ ]:
Y_new=Y+Y1

In [ ]:
lenofimage = len(images_new)

In [ ]:
X=[]
y=[]


X.append(images_new)
y.append(Y_new)


In [ ]:
X= np.array(X).reshape(lenofimage,-1)

In [ ]:
X.shape

(32079, 14641)

In [ ]:
X = X/255.0

In [ ]:
X[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
y=np.array(y)

In [ ]:
y.shape

(1, 32079)

In [ ]:
y2=y.reshape([32079,])

In [ ]:
y2.shape

(32079,)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y2)

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='linear',gamma='auto')
svc.fit(X_train, y_train)

SVC(gamma='auto', kernel='linear')

In [ ]:
y3 = svc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy on test data is",accuracy_score(y_test,y3))

Accuracy on test data is 0.5213216957605985


In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier(criterion='entropy',max_depth=30,random_state=0).fit(X_train, y_train)
y4 = clf.predict(X_test)
from sklearn.metrics import accuracy_score
print("Accuracy of DT on test data is",accuracy_score(y_test,y4))

Accuracy of DT on test data is 0.6359102244389028


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=50,max_depth=30,criterion='gini').fit(X_train, y_train)
y5 = rf.predict(X_test)
from sklearn.metrics import accuracy_score
print("Accuracy of RF on test data is",accuracy_score(y_test,y5))

Accuracy of RF on test data is 0.8024937655860349


In [ ]:
import pandas as pd
result = pd.DataFrame({'original' : y_test,'predicted' : y5})
result

,original,predicted
0,No,No
1,No,No
2,No,No
3,Yes,Yes
4,Yes,Yes
...,...,...
8015,Yes,Yes
8016,Yes,Yes
8017,Yes,Yes
8018,No,No
